<a href="https://colab.research.google.com/github/Hamidkhandata/HamidProjectDS10/blob/main/HamidChatbotNLP_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ Step 1: Install Required Libraries
!pip install -q PyMuPDF faiss-cpu sentence-transformers transformers gradio langchain langchain-community

In [ ]:
# ✅ Step 2: Upload One PDF File
from google.colab import files
uploaded_file = files.upload()

Saving Priority transformation initiatives KP_Final Minutes.pdf to Priority transformation initiatives KP_Final Minutes (1).pdf


In [ ]:
import fitz  # PyMuPDF

In [ ]:
filename = list(uploaded_file.keys())[0]
with fitz.open(filename) as doc:
    full_text = ""
    for page in doc:
        full_text += page.get_text()

In [ ]:
# ✅ Step 3: Split Text into Chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
docs = splitter.split_documents([Document(page_content=full_text)])

In [ ]:
# ✅ Step 4: Embed Chunks Using Sentence Transformers
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = FAISS.from_documents(docs, embedding_model)

In [ ]:
# ✅ Step 5: Load HuggingFace Language Model (Flan-T5)
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

In [ ]:
# ✅ Step 6: Answer Generation Function
def generate_answer(question, context):
    prompt = f"Context: {context}\n\nQuestion: {question}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    outputs = model.generate(**inputs, max_length=512)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# ✅ Step 7: Gradio Chat Interface
import gradio as gr

In [ ]:
def chat_with_doc(message, history):
    docs = db.similarity_search(message, k=3)
    context = "\n\n".join([doc.page_content for doc in docs])
    return generate_answer(message, context)

In [ ]:
gr.ChatInterface(fn=chat_with_doc, title="Single Document Chatbot (Free NLP)").launch()

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://012ed0d2d7c9af6863.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
